In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import anndata as ad
from anndata import AnnData
import scanpy as sc


In [ ]:
adata_exp0 = ad.read_h5ad('C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/spatial/counts_CPc_exp0_BA28.h5ad')

In [ ]:
adata_Yao = ad.read_h5ad(
    'C:/Users/mdichgan/Documents/Helmholtz/send_to_Jakob/sc/Yao_150kcells_subsample_with_annotations_sparse_subset.h5ad')

In [76]:
adata_Yao.obs["celltype"] = adata_Yao.obs["label"]

In [ ]:
sc.pp.normalize_total(adata_exp0)
sc.pp.normalize_total(adata_Yao)
# sc.pp.log1p(adata_exp0)
# sc.pp.log1p(adata_Yao)
adata_exp0.layers["lognorm"] = adata_exp0.X
adata_Yao.layers["lognorm"] = adata_Yao.X

In [ ]:
def get_modality_ratio_score(adata_st: AnnData, adata_sc: AnnData, obs_key: str = "celltype", k: int = 45,ct_filter_factor: float = 2):
    adata_st.obs["modality"] = "spatial"
    adata_sc.obs["modality"] = "sc"
    adata = ad.concat([adata_st, adata_sc], join='inner')  

    # Set counts to log norm data
    adata.X = adata.layers["lognorm"]
    
    # Calculate PCA (Note: we could also think about pca per cell type...)
    assert (adata.obsm is None) or ('X_pca' not in adata.obsm), "PCA already exists."
    sc.tl.pca(adata)
    
    # get cell type groups
    sc_cts = set(adata_sc.obs["celltype"].cat.categories)
    st_cts = set(adata_st.obs["celltype"].cat.categories)
    shared_cts = list(sc_cts.intersection(st_cts))         

    # Get ratio per shared cell type
    df = pd.DataFrame(columns=["cell_id","celltype","ratio","exp_val","score"])
    for ct in shared_cts:
        enough_cells = (adata.obs.loc[adata.obs[obs_key]==ct,"modality"].value_counts() > (ct_filter_factor * k)).all()     #nochmal: wieso ct_fil?
        if enough_cells:
            a = adata[adata.obs[obs_key]==ct]
            exp_val = (a.obs.loc[a.obs["modality"]=="sc"].shape[0])/a.obs.shape[0]
            sc.pp.neighbors(a,n_neighbors=k)
            G = nx.Graph(incoming_graph_data=a.obsp["connectivities"])
            nx.set_node_attributes(G, {i:a.obs["modality"].values[i] for i in range(G.number_of_nodes())}, "modality")   

            ct_df = np.zeros(a.obs.shape[0])
            f = np.vectorize(lambda x: x/exp_val if x>0 and x<=exp_val else x/(exp_val-1)+1/(1-exp_val))
            i = 0
            for cell in G.nodes():
                ct_df[i] = sum(1 for neighbor in G.neighbors(cell) if G.nodes[cell]["modality"]=="sc")  #number_modality_sc
                ct_df[i] = ct_df[i]/G.degree(cell)  
                i += 1
                  #ratio: number modality sc / total cells
                # if total_cells != 0:              #0 nicht möglich? immer k?
                #     ratio = number_modality_sc / total_cells
                # else:
                #     ratio = 0        #no neighbors
                #df.append({"cell_id": "cell_id", "celltype": ct, "ratio": ratio, "exp_val":exp_val, "score": score})  #wie machen mit cell_id? ineffizient
                #df aus liste an listen
            daf = pd.DataFrame({"cell_id": a.obs["cell_id"], "celltype": ct, "ratio":ct_df, "exp_val":exp_val, "score":f(ct_df)})
    
    df = pd.concat([df,daf])
    return df


In [ ]:
adata_Yao.obs = adata_Yao.obs.reset_index().rename(columns={'index':'cell_id'})
adata_Yao.obs

In [79]:
adata_Yao.obs["celltype"]

0        L2/3 IT RSP
1         Lamp5 Lhx6
2        L2/3 IT CTX
3                Vip
4        L5/6 IT CTX
            ...     
24504            Vip
24505          Oligo
24506      L6 CT CTX
24507        L6b CTX
24508          Pvalb
Name: celltype, Length: 24509, dtype: category
Categories (57, object): ['Astro', 'CA1', 'CA1-ProS', 'CA2-IG-FC', ..., 'Sst', 'Sst Chodl', 'VLMC', 'Vip']

In [74]:
adata_exp0.obs.loc[adata_exp0.obs["cell_id"]==1]

,cell_id,ct_majority,ct_majority_cert,celltype,n_counts,n_genes,area,modality
Cell_3451,1,Cck INH,1.0,Cck INH,102.0,33,1253.0,spatial


In [ ]:
#test get_modality_ratio_score
get_modality_ratio_score(adata_exp0, adata_Yao)       